In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import datasets

# Upload texts from Scikit-Learn database

newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

# массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target

In [2]:
y = newsgroups.target
X = newsgroups.data

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

In [4]:
# vectorize newsgroups.data 

# Error "fit_transform() missing 1 required positional argument: 'raw_documents'" resolves by putting () after TfidfVectorizer

X_vect = vectorizer.fit_transform(X)

X_vect

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [5]:
# calibrate minimal parameter for SVM

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

grid = {'C': np.power(10.0, np.arange(-5, 6))} # dict sets the grid of parameter for iterating
cv = KFold(n_splits=5, shuffle=True, random_state=241) #cross-validation
clf = SVC(kernel='linear', random_state=241) 
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)

# train the model on the vectorized data

gs.fit(X_vect, y) 

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=241,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [6]:
# Our best parameter for SVM

best_C = gs.best_estimator_.C
print(best_C)

1.0


In [7]:
# train the model on the vectorized data with optimal C parameter

clf1 = SVC(kernel='linear', random_state=241, C = best_C)

clf1.fit(X_vect,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=241,
    shrinking=True, tol=0.001, verbose=False)

In [8]:
# searching for the highest coefficients, i.e. the words with highest wieghts

# Нужно получить наибольшие 10 весов из обученного классификатора (SVC), узнать их индексы.
# Потом по этим индексам узнать с помощью vectorizer.get_feature_names() каким словам принадлежат эти индексы. 
# Проблема в там, что поле coef_ - разряженная матрица, к тому же неотсортированная по убыванию или возрастанию.
# Необходимо привести её к обычному виду, а затем упорядочить по возрастанию или убыванию. 
# При этом упорядочить надо так, чтобы индексы не поменялись.

# Сконвертируем результат в Pandas и потом уже из него выбирем все, что требуется

coefs = clf1.coef_

q = pd.DataFrame(coefs.toarray()).transpose()

# Чтобы все правильно отсортировать со значениями по модулю нужно добавить функцию abs().

top10=abs(q).sort_values([0], ascending=False).head(10) 

In [9]:
# Which 10 words have the highest modulo weight

indices=[]
indices=top10.index

words=[]
for i in indices:
    feature_mapping = vectorizer.get_feature_names()
    words.append(feature_mapping[i])

print(sorted(words))

['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion', 'sci', 'sky', 'space']
